In [169]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [170]:
def marital_status_to_num(val):
    if val == 'M':
        return 0
    elif val == 'S':
        return 1
    elif val == 'D':
        return 2
    elif val == 'W':
        return 3
    else:
        return np.nan

def marital_status_to_str(val):
    if val == 0:
        return 'married'
    elif val == 1:
        return 'single'
    elif val == 2:
        return 'divorced'
    elif val == 3:
        return 'widowed'
    else:
        return np.nan

data_dict = {
    'data': {
        'rename': {
            'ID Number': 'id',
            'Lifetime HC': 'cum_donation',
            'Email Present': 'has_email',
            'BusPhone Present': 'has_business_phone',
            'Grad Year': 'grad_year',
            'Marital Status': 'marital_status',
            'SpouseID Present': 'has_spousal_record',
            'JobTitle Present': 'has_job_title',
            'VarsityAth Present': 'has_activity_athlete',
            'StudGovt Present': 'has_activity_government',
            'OtherStudActs Present': 'has_activity_other',
            'Greek Present': 'has_activity_greek',
            'Prefix is Mr.': 'is_mr',
            'Prefix is Ms.': 'is_ms',
            'Prefix is Dr.': 'is_dr',
            'Prefix is Mrs.': 'is_mrs'
        },
        'dtype': {
            'ID Number': str,
            'Lifetime HC': float,
            'Grad Year': int,
            'Marital Status': str
        }
    },
    'dummy': {
        'list': [
            'cum_range',
            'grad_decade',
            'imp_marital_status',
        ],
        'prefix': {
            'cum_range': 'bin_cum_range',
            'grad_decade': 'bin_grad_decade',
            'imp_marital_status': 'bin_imp_marital_status'
        },
        'giving': {
            'bins': [
                0.00,
                1.00,
                1000.00,
                10000.00,
                25000.00,
                50000.00,
                100000.00,
                250000.00,
                500000.00,
                1000000.00,
                2500000.00,
                5000000.00,
                10000000.00,
                15000000.00
            ],
            'labels': [
                '$0',
                '$1-$999.99',
                '$1K-$9.99K',
                '$10K-$24.99K',
                '$25K-$49.99K',
                '$50K-$99.99K',
                '$100K-$249.99K',
                '$250K-$499.99K',
                '$500K-$999.99K',
                '$1M-$2.49M',
                '$2.5M-$4.99M',
                '$5M-$9.99M',
                '$10M-$14.99M'
            ]
        },
        'grad': {
            'bins': [
                1900,
                1910,
                1920,
                1930,
                1940,
                1950,
                1960,
                1970,
                1980,
                1990,
                2000,
                2010,
                2020
            ],
            'labels': [
                '1900s',
                '1910s',
                '1920s',
                '1930s',
                '1940s',
                '1950s',
                '1960s',
                '1970s',
                '1980s',
                '1990s',
                '2000s',
                '2010s'
            ]
        }
    },
    'cols': {
        'bools': [
            'has_email',
            'has_business_phone',
            'has_spousal_record',
            'has_job_title',
            'has_activity_athlete',
            'has_activity_government',
            'has_activity_other',
            'has_activity_greek',
            'is_mr',
            'is_ms',
            'is_dr',
            'is_mrs',
            'has_donated'
        ],
        'bin_grad': [
            'bin_grad_decade_1900s',
            'bin_grad_decade_1910s',
            'bin_grad_decade_1920s',
            'bin_grad_decade_1930s',
            'bin_grad_decade_1940s',
            'bin_grad_decade_1950s',
            'bin_grad_decade_1960s',
            'bin_grad_decade_1970s',
            'bin_grad_decade_1980s',
            'bin_grad_decade_1990s',
            'bin_grad_decade_2000s',
            'bin_grad_decade_2010s'
        ],
        'bin_giving': [
            'bin_cum_range_$0',
            'bin_cum_range_$1-$999.99',
            'bin_cum_range_$1K-$9.99K',
            'bin_cum_range_$10K-$24.99K',
            'bin_cum_range_$25K-$49.99K',
            'bin_cum_range_$50K-$99.99K',
            'bin_cum_range_$100K-$249.99K',
            'bin_cum_range_$250K-$499.99K',
            'bin_cum_range_$500K-$999.99K',
            'bin_cum_range_$1M-$2.49M',
            'bin_cum_range_$2.5M-$4.99M',
            'bin_cum_range_$5M-$9.99M',
            'bin_cum_range_$10M-$14.99M'
        ],
        'bin_marital': [
            'bin_imp_marital_status_divorced',
            'bin_imp_marital_status_married',
            'bin_imp_marital_status_single',
            'bin_imp_marital_status_widowed'
        ]
    }
}

In [171]:
project_dir = os.path.join(os.path.abspath(''),os.pardir)
data_dir = os.path.join(project_dir,'data')
data_raw_dir = os.path.join(data_dir,'raw')

file_list = []

for file in os.scandir(data_raw_dir):
    file_list.append(os.path.join(data_raw_dir,file.name))

df = pd.read_excel(io=file_list[0],
                   sheet_name='Sheet1',
                   dtype=data_dict['data']['dtype'])
df = df.rename(columns=data_dict['data']['rename'])

/home/rob/.local/share/virtualenvs/assignment2-vA0J_QZc/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [172]:
# drop irrelevant columns
df = df.drop(columns='id')

In [173]:
# create a logistic column version of cum_donation_value called has_donated as an additional target variable
df['has_donated'] = np.where(df['cum_donation']>0.0,True,False)

y_log = df[['has_donated']]
y_lin = df[['cum_donation']]

In [174]:
# imputing
df['marital_status'] = df['marital_status'].apply(marital_status_to_num)
df.loc[df['marital_status'].isnull(),'m_marital_status'] = 1
df.loc[df['marital_status'].notnull(),'m_marital_status'] = 0

df_marital_status = df[['m_marital_status']]
df = df.drop(columns='m_marital_status')

imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df),columns=df.columns)
df = df.rename(columns={'marital_status': 'imp_marital_status'})
df = pd.concat((df,df_marital_status),axis=1)

df['imp_marital_status'] = df['imp_marital_status'].round().astype(int)
df['imp_marital_status'] = df['imp_marital_status'].apply(marital_status_to_str)
df[data_dict['cols']['bools']] = df[data_dict['cols']['bools']].astype(int)

In [175]:
# binning
df.sort_values('cum_donation',inplace=True)
df['cum_range'] = \
    pd.cut(x=df['cum_donation'],
           bins=data_dict['dummy']['giving']['bins'],
           labels=data_dict['dummy']['giving']['labels'],
           right=False)
df.sort_values('grad_year',inplace=True)
df['grad_decade'] = pd.cut(x=df['grad_year'],
                           bins=data_dict['dummy']['grad']['bins'],
                           labels=data_dict['dummy']['grad']['labels'],
                           right=False)
df.sort_index(inplace=True)

In [176]:
# dummying
dummy_filter = df[data_dict['dummy']['list']]
dummy_df = pd.get_dummies(data=dummy_filter,
                          prefix=data_dict['dummy']['prefix'],
                          columns=data_dict['dummy']['list'])
df = pd.concat(([df,dummy_df]),axis=1)

df = df.drop(columns=['imp_marital_status','cum_range','grad_decade'])

# Create four flat files to take test the effectiveness of the binning process
# 00 - cum_donation(float), grad_year(int)
# 01 - cum_donation(float), grad_year(bin)
# 10 - cum_donation(bin), grad_year(int)
# 11 - cum_donation(bin), grad_year(bin)

X_dict = {
    'log': {
        'X_00': {
            'df': df.drop(columns=['has_donated'] \
                                + data_dict['cols']['bin_grad'] \
                                + data_dict['cols']['bin_giving']).copy(),
            'chi': list()
        },
        'X_01': {
            'df': df.drop(columns=['has_donated','grad_year'] \
                                + data_dict['cols']['bin_giving']).copy(),
            'chi': list()
        },
        'X_10': {
            'df': df.drop(columns=['has_donated','cum_donation'] \
                                + data_dict['cols']['bin_grad']).copy(),
            'chi': list()
        },
        'X_11': {
            'df': df.drop(columns=['has_donated',
                                   'cum_donation',
                                   'grad_year']).copy(),
            'chi': list()
        }
    },
    'lin': {
        'X_00': {
            'df': df.drop(columns=['cum_donation'] \
                                + data_dict['cols']['bin_grad'] \
                                + data_dict['cols']['bin_giving']).copy(),
            'predictors': list()
        },
        'X_01': {
            'df': df.drop(columns=['has_donated','grad_year'] \
                                + data_dict['cols']['bin_giving']).copy(),
            'predictors': list()
        }
    }
}

In [177]:
# Logistic Feature Selection, Chi-Squared

test = SelectKBest(score_func=chi2,k='all')
np.set_printoptions(precision=3)

for k,v in X_dict['log'].items():
    chi_scores = test.fit(X_dict['log'][k]['df'], y_log)
    chi_scores_list = list(zip(list(X_dict['log'][k]['df'].columns), chi_scores.scores_))
    chi_scores_df = pd.DataFrame(chi_scores_list,columns=['predictor','significance'])
    chi_scores_df.sort_values('significance',ascending=False,inplace=True)
    X_dict['log'][k]['chi'] = list(chi_scores_df['predictor'][chi_scores_df['significance'] > 3.8])
    print(X_dict['log'][k]['chi'])

['cum_donation', 'm_marital_status', 'has_spousal_record', 'bin_imp_marital_status_single', 'has_business_phone', 'bin_imp_marital_status_married', 'has_email', 'is_mrs', 'has_job_title', 'grad_year', 'bin_imp_marital_status_widowed', 'is_ms', 'has_activity_greek', 'has_activity_government', 'is_dr', 'has_activity_athlete', 'bin_imp_marital_status_divorced', 'has_activity_other', 'is_mr']
['cum_donation', 'm_marital_status', 'has_spousal_record', 'bin_imp_marital_status_single', 'has_business_phone', 'bin_grad_decade_2000s', 'bin_grad_decade_2010s', 'bin_imp_marital_status_married', 'bin_grad_decade_1960s', 'has_email', 'is_mrs', 'has_job_title', 'bin_grad_decade_1970s', 'bin_grad_decade_1950s', 'bin_imp_marital_status_widowed', 'is_ms', 'has_activity_greek', 'has_activity_government', 'bin_grad_decade_1940s', 'bin_grad_decade_1990s', 'bin_grad_decade_1980s', 'is_dr', 'has_activity_athlete', 'bin_imp_marital_status_divorced', 'has_activity_other', 'is_mr']
['bin_cum_range_$0', 'bin_cum

In [178]:
# Logistic Feature Selection, RFE | FFS

# Assignment 2

## Introduction

## Data Exploration

## Data-Preparation

### Identifying Significant Features

### Imputing and Variable Creation

## Data Modeling

### Scaling

## Model Evaluation

## Conclusion